In [1]:
#importation
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from itertools import islice
import sys  
sys.path.insert(0, '../scripts')
import clean_data
import os

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
#loading data and displaying columns
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
#data_path = os.path.join(parent_dir, "data", "Week1_challenge_data_source.csv")
data_path2 = os.path.join(parent_dir, "data", "user_engage.csv")
data_path3 =  os.path.join(parent_dir, "data", "user_exp.csv")
#db = pd.read_csv(data_path)
user_engagement = pd.read_csv(data_path2)
user_exp = pd.read_csv(data_path3)
user_engagement.drop('Unnamed: 0', inplace = True, axis =1)
user_engagement.head()

,Customer Number,Session Frequency,Session Duration,Total Data Volume,Google(UL/DL),Email(UL/DL),Gaming(UL/DL),Youtube(UL/DL),Netflix(UL/DL),Social(UL/DL)
0,3.360100e+10,1,116720.0,8.786906e+08,2905912.0,3701304.0,292426453.0,18355943.0,17855187.0,1570185.0
1,3.360100e+10,1,181230.0,1.568596e+08,4414096.0,937385.0,609920783.0,39359124.0,35565545.0,1933278.0
2,3.360100e+10,1,134969.0,5.959665e+08,10229119.0,3363124.0,229980251.0,34425237.0,23751202.0,1726277.0
3,3.360101e+10,1,49878.0,4.223207e+08,11811761.0,2070983.0,810387875.0,36534765.0,15092588.0,657493.0
4,3.360101e+10,2,37104.0,1.457411e+09,7748843.0,2110349.0,531237049.0,34222253.0,17539799.0,912788.0


In [4]:
usr_eng = user_engagement[['Customer Number' ,'Session Frequency','Session Duration','Total Data Volume']]
column = ['Customer Number' ,'Session Frequency','Session Duration','Total Data Volume']
for c in column:
    usr_eng[c]=user_engagement[c]

usr_eng.head()

,Customer Number,Session Frequency,Session Duration,Total Data Volume
0,3.360100e+10,1,116720.0,8.786906e+08
1,3.360100e+10,1,181230.0,1.568596e+08
2,3.360100e+10,1,134969.0,5.959665e+08
3,3.360101e+10,1,49878.0,4.223207e+08
4,3.360101e+10,2,37104.0,1.457411e+09


In [5]:
to_scale = usr_eng[['Session Frequency','Session Duration','Total Data Volume']].values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
scaled = min_max_scaler.fit_transform(to_scale)
normalized = pd.DataFrame(scaled, columns=['Session Frequency','Session Duration','Total Data Volume'])

In [6]:
from sklearn.cluster import KMeans
KMean_clust= KMeans(n_clusters=3, init='k-means++',max_iter=1000)
KMean_clust.fit(normalized)

KMeans(max_iter=1000, n_clusters=3)

In [7]:
normalized['Cluster']= KMean_clust.labels_

metrics = usr_eng[['Session Frequency','Session Duration','Total Data Volume']]
metrics['Cluster']=KMean_clust.labels_
metrics

,Session Frequency,Session Duration,Total Data Volume,Cluster
0,1,116720.0,8.786906e+08,1
1,1,181230.0,1.568596e+08,1
2,1,134969.0,5.959665e+08,1
3,1,49878.0,4.223207e+08,1
4,2,37104.0,1.457411e+09,0
...,...,...,...,...
106851,1,8810.0,7.146416e+08,1
106852,1,140988.0,4.803073e+08,1
106853,1,877385.0,2.321240e+08,1
106854,1,253030.0,5.962878e+08,1


In [8]:
clus_mean = metrics.groupby('Cluster').mean()
clus_mean

,Session Frequency,Session Duration,Total Data Volume
Cluster,,,
0,2.177390,213764.104121,1.126854e+09
1,1.042627,107015.903747,4.955583e+08
2,4.254388,570738.252658,2.283464e+09


In [9]:
centroids = KMean_clust.cluster_centers_
centroids = min_max_scaler.inverse_transform(centroids)
print(centroids)
usr_eng['Engagement Score'] = np.linalg.norm(usr_eng[['Session Frequency', 'Session Duration','Total Data Volume']].sub(np.array(centroids[0])), axis=1)
usr_eng

[[2.17737456e+00 2.13705666e+05 1.12704192e+09]
 [1.04276883e+00 1.07040102e+05 4.95576532e+08]
 [4.25345850e+00 5.70616611e+05 2.28311168e+09]]


,Customer Number,Session Frequency,Session Duration,Total Data Volume,Engagement Score
0,3.360100e+10,1,116720.0,8.786906e+08,2.483514e+08
1,3.360100e+10,1,181230.0,1.568596e+08,9.701823e+08
2,3.360100e+10,1,134969.0,5.959665e+08,5.310754e+08
3,3.360101e+10,1,49878.0,4.223207e+08,7.047212e+08
4,3.360101e+10,2,37104.0,1.457411e+09,3.303691e+08
...,...,...,...,...,...
106851,3.379000e+10,1,8810.0,7.146416e+08,4.124003e+08
106852,3.379000e+10,1,140988.0,4.803073e+08,6.467346e+08
106853,3.197020e+12,1,877385.0,2.321240e+08,8.949182e+08
106854,3.370000e+14,1,253030.0,5.962878e+08,5.307541e+08


In [20]:
user_exp.head()

,Customer Number,Avg Bearer TP(UL/DL),Avg RTT(UL/DL),TCP Retrans(UL/DL),No. of Handset
0,3.360100e+10,76.0,46.000000,2.156957e+07,1
1,3.360100e+10,99.0,31.000000,2.156957e+07,1
2,3.360100e+10,97.0,127.458589,2.156957e+07,1
3,3.360101e+10,248.0,84.000000,7.607247e+05,1
4,3.360101e+10,56844.0,119.000000,3.094040e+07,1


In [21]:
to_scale = user_exp[['Avg Bearer TP(UL/DL)','Avg RTT(UL/DL)','TCP Retrans(UL/DL)']].values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
scaled = min_max_scaler.fit_transform(to_scale)
normalized = pd.DataFrame(scaled, columns=['Avg Bearer TP(UL/DL)','Avg RTT(UL/DL)','TCP Retrans(UL/DL)'])
KMean_clust= KMeans(n_clusters=3, init='k-means++',max_iter=1000)
KMean_clust.fit(normalized)
metric=user_exp[['Avg Bearer TP(UL/DL)','Avg RTT(UL/DL)','TCP Retrans(UL/DL)']]
metric['Cluster']=KMean_clust.labels_
metric

,Avg Bearer TP(UL/DL),Avg RTT(UL/DL),TCP Retrans(UL/DL),Cluster
0,76.0,46.000000,2.156957e+07,1
1,99.0,31.000000,2.156957e+07,1
2,97.0,127.458589,2.156957e+07,1
3,248.0,84.000000,7.607247e+05,1
4,56844.0,119.000000,3.094040e+07,0
...,...,...,...,...
106851,10365.0,52.000000,2.180450e+05,1
106852,116.0,40.000000,2.156957e+07,1
106853,1.0,127.458589,2.156957e+07,1
106854,33.0,127.458589,2.156957e+07,1


In [22]:
clus_mean = metric.groupby('Cluster').mean()
clus_mean

,Avg Bearer TP(UL/DL),Avg RTT(UL/DL),TCP Retrans(UL/DL)
Cluster,,,
0,67407.383175,183.708397,3.962415e+07
1,4820.382024,167.192941,2.144235e+07
2,209915.282109,397.580360,2.408296e+08


In [17]:
usr_eng.columns.tolist()

['Customer Number',
 'Session Frequency',
 'Session Duration',
 'Total Data Volume',
 'Engagement Score']

In [21]:
centroids = KMean_clust.cluster_centers_
centroids = min_max_scaler.inverse_transform(centroids)

satis=user_exp
satis['Experience Score'] = np.linalg.norm( user_exp[['Avg Bearer TP(UL/DL)','Avg RTT(UL/DL)','TCP Retrans(UL/DL)']].sub(np.array(centroids[0])), axis=1)

# retreive engagement scores
satis['Engagement Score']=usr_eng['Engagement Score'].values
col = satis.loc[: , "Engagement Score":"Experience Score"]
satis['Satisfaction'] = user_exp[["Engagement Score","Experience Score"]].mean(axis=1)
satis=satis[["Engagement Score","Experience Score",'Satisfaction']]
satis.nlargest(10,'Satisfaction')


,Engagement Score,Experience Score,Satisfaction
6437,7.719191e+09,9.926243e+08,4.355908e+09
66860,1.078445e+09,7.227167e+09,4.152806e+09
92923,7.387738e+09,8.768352e+08,4.132286e+09
13180,7.372602e+09,8.648591e+08,4.118730e+09
13526,6.844131e+09,8.901919e+08,3.867161e+09
76363,6.764070e+09,9.596889e+08,3.861880e+09
37052,6.578822e+09,1.037395e+09,3.808108e+09
34123,5.257896e+08,6.814818e+09,3.670304e+09
42366,1.045118e+09,6.247857e+09,3.646487e+09
63028,6.181461e+09,9.960443e+08,3.588752e+09


In [23]:
from sklearn.linear_model import LinearRegression

x = satis[['Engagement Score','Experience Score']]
y = satis['Satisfaction']
x, y = np.array(x), np.array(y)
model = LinearRegression().fit(x, y)
r_sq = model.score(x, y)
print('coefficient determination:', r_sq)
print('intercept:', model.intercept_)
print('slope:', model.coef_)
y_pred = model.predict(x)
print('predicted response:', y_pred, sep='\n')

coefficient determination: 1.0
intercept: -1.5497207641601562e-06
slope: [0.5 0.5]
predicted response:
[6.76911871e+08 1.03782733e+09 8.18273909e+08 ... 1.00019529e+09
 8.18113237e+08 1.04668043e+09]


In [24]:
x = satis[['Engagement Score','Experience Score']].values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
normalized = pd.DataFrame(x_scaled,columns=['Engagement Score','Experience Score'])
KMean_clust= KMeans(n_clusters=2, init='k-means++',max_iter=1000)
KMean_clust.fit(normalized)
satis['Cluster']=KMean_clust.labels_
satis

,Engagement Score,Experience Score,Satisfaction,Cluster
0,2.483514e+08,1.105472e+09,6.769119e+08,0
1,9.701823e+08,1.105472e+09,1.037827e+09,1
2,5.310754e+08,1.105472e+09,8.182739e+08,0
3,7.047212e+08,1.126281e+09,9.155012e+08,1
4,3.303691e+08,1.096102e+09,7.132353e+08,0
...,...,...,...,...
106851,4.124003e+08,1.126824e+09,7.696121e+08,0
106852,6.467346e+08,1.105472e+09,8.761035e+08,1
106853,8.949182e+08,1.105472e+09,1.000195e+09,1
106854,5.307541e+08,1.105472e+09,8.181132e+08,0


In [26]:
satis.groupby('Cluster')[['Engagement Score','Experience Score']].mean()

,Engagement Score,Experience Score
Cluster,,
0,3.473984e+08,1.103350e+09
1,8.516750e+08,1.106862e+09


In [ ]:
import sqlite3
con = sqlite3.connect ('telecom.db')
#print('The python version is...',sqlite3.version)

cursor = con.cursor ()
try:
    cur.execute ('create table user_satisfaction_level (user_id, eng_score number, exp_score number, Satisfaction)')
except :
    print('Table exists.... continuing')

satisfactionDF=satisfactionDF.rename(columns={"Eng Score": "eng_score", "Exp Score": "exp_score"})
satisfactionDF['user_id']=satisfactionDF.index
satisfactionDF[['user_id','eng_score','exp_score','Satisfaction']].to_sql('user_satisfaction_level', con, if_exists='replace', index=False)

satisfactionDF